In [1]:
import sys
import numpy as np
import math
import random

import gym
from envs.maze_view_2d import MazeView2D
import envs.maze_env

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
new_maze = np.array([
    [6,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,12],
    [7,15,10,10,10,10,10,10,10,10,10,10,10,10,10,10,15,13],
    [7,5,0,0,7,13,0,0,7,5,0,0,7,5,0,0,5,13],
    [7,5,0,0,7,13,0,0,7,5,0,0,7,5,0,0,5,13],
    [7,5,0,0,7,13,0,0,7,5,0,0,7,5,0,0,5,13],
    [7,5,0,0,7,13,0,0,7,5,0,0,7,5,0,0,5,13],
    [7,5,0,0,7,13,0,0,7,5,0,0,7,5,0,0,5,13],
    [7,5,0,0,7,13,0,0,7,5,0,0,7,5,0,0,5,13],
    [7,5,10,10,15,15,10,10,15,15,10,10,15,15,10,10,5,13],
    [7,5,10,10,15,15,10,10,15,15,10,10,15,15,10,10,5,13],
    [7,5,0,0,7,13,0,0,7,5,0,0,7,5,0,0,5,13],
    [7,5,0,0,7,13,0,0,7,5,0,0,7,5,0,0,5,13],
    [7,5,0,0,7,13,0,0,7,5,0,0,7,5,0,0,5,13],
    [7,5,0,0,7,13,0,0,7,5,0,0,7,5,0,0,5,13],
    [7,5,0,0,7,13,0,0,7,5,0,0,7,5,0,0,5,13],
    [7,5,0,0,7,13,0,0,7,5,0,0,7,5,0,0,5,13],
    [7,15,10,10,10,10,10,10,10,10,10,10,10,10,10,10,15,13],
    [3,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,9],
])
#np.save('./maze_warehouse',new_maze.T)

In [2]:
maze = MazeView2D(screen_size= (500, 500), maze_file_path='D:/notebooks/MMAI 845/gym-maze/gym_maze/envs/maze_samples/maze_warehouse.npy', num_portals=0, has_loops=False , num_picks=4 )
maze.reset_robot()
log = maze.update()

((15, 1), (13, 7), (5, 6), (16, 6))
test self.num_picks =  4


In [3]:
maze.move_robot('S')
log = maze.update()

In [22]:
maze.move_robot('E')
log = maze.update()

In [24]:
picks = maze.maze.get_pick_all()
picks

{(15, 1): <envs.maze_view_2d.Pick at 0x1db75cf27c8>,
 (13, 7): <envs.maze_view_2d.Pick at 0x1db75cf27c8>,
 (5, 6): <envs.maze_view_2d.Pick at 0x1db75cf27c8>,
 (16, 6): <envs.maze_view_2d.Pick at 0x1db75cf27c8>}

In [3]:
import sys
import numpy as np
import math
import random

import gym
import gym_maze


def simulate():

    # Instantiating the learning related parameters
    learning_rate = get_learning_rate(0)
    explore_rate = get_explore_rate(0)
    discount_factor = 0.99

    num_streaks = 0

    # Render tha maze
    env.render()

    for episode in range(NUM_EPISODES):

        # Reset the environment
        obv = env.reset()

        # the initial state
        state_0 = state_to_bucket(obv)
        total_reward = 0

        for t in range(MAX_T):

            # Select an action
            action = select_action(state_0, explore_rate)

            # execute the action
            obv, reward, done, _ = env.step(action)

            # Observe the result
            state = state_to_bucket(obv)
            total_reward += reward

            # Update the Q based on the result
            best_q = np.amax(q_table[state])
            q_table[state_0 + (action,)] += learning_rate * (reward + discount_factor * (best_q) - q_table[state_0 + (action,)])

            # Setting up for the next iteration
            state_0 = state

            # Print data
            if DEBUG_MODE == 2:
                print("\nEpisode = %d" % episode)
                print("t = %d" % t)
                print("Action: %d" % action)
                print("State: %s" % str(state))
                print("Reward: %f" % reward)
                print("Best Q: %f" % best_q)
                print("Explore rate: %f" % explore_rate)
                print("Learning rate: %f" % learning_rate)
                print("Streaks: %d" % num_streaks)
                print("")

            elif DEBUG_MODE == 1:
                if done or t >= MAX_T - 1:
                    print("\nEpisode = %d" % episode)
                    print("t = %d" % t)
                    print("Explore rate: %f" % explore_rate)
                    print("Learning rate: %f" % learning_rate)
                    print("Streaks: %d" % num_streaks)
                    print("Total reward: %f" % total_reward)
                    print("")

            # Render tha maze
            if RENDER_MAZE:
                env.render()

            if env.is_game_over():
                sys.exit()

            if done:
                print("Episode %d finished after %f time steps with total reward = %f (streak %d)."
                      % (episode, t, total_reward, num_streaks))

                if t <= SOLVED_T:
                    num_streaks += 1
                else:
                    num_streaks = 0
                break

            elif t >= MAX_T - 1:
                print("Episode %d timed out at %d with total reward = %f."
                      % (episode, t, total_reward))

        # It's considered done when it's solved over 120 times consecutively
        if num_streaks > STREAK_TO_END:
            break

        # Update parameters
        explore_rate = get_explore_rate(episode)
        learning_rate = get_learning_rate(episode)


def select_action(state, explore_rate):
    # Select a random action
    if random.random() < explore_rate:
        action = env.action_space.sample()
    # Select the action with the highest q
    else:
        action = int(np.argmax(q_table[state]))
    return action


def get_explore_rate(t):
    return max(MIN_EXPLORE_RATE, min(0.8, 1.0 - math.log10((t+1)/DECAY_FACTOR)))


def get_learning_rate(t):
    return max(MIN_LEARNING_RATE, min(0.8, 1.0 - math.log10((t+1)/DECAY_FACTOR)))


def state_to_bucket(state):
    bucket_indice = []
    for i in range(len(state)):
        if state[i] <= STATE_BOUNDS[i][0]:
            bucket_index = 0
        elif state[i] >= STATE_BOUNDS[i][1]:
            bucket_index = NUM_BUCKETS[i] - 1
        else:
            # Mapping the state bounds to the bucket array
            bound_width = STATE_BOUNDS[i][1] - STATE_BOUNDS[i][0]
            offset = (NUM_BUCKETS[i]-1)*STATE_BOUNDS[i][0]/bound_width
            scaling = (NUM_BUCKETS[i]-1)/bound_width
            bucket_index = int(round(scaling*state[i] - offset))
        bucket_indice.append(bucket_index)
    return tuple(bucket_indice)


if __name__ == "__main__":

    # Initialize the "maze" environment
    env = gym.make("maze-warehouse-picks-v0")

    '''
    Defining the environment related constants
    '''
    # Number of discrete states (bucket) per state dimension
    MAZE_SIZE = tuple((env.observation_space.high + np.ones(env.observation_space.shape)).astype(int))
    NUM_BUCKETS = MAZE_SIZE  # one bucket per grid

    # Number of discrete actions
    NUM_ACTIONS = env.action_space.n  # ["N", "S", "E", "W"]
    # Bounds for each discrete state
    STATE_BOUNDS = list(zip(env.observation_space.low, env.observation_space.high))

    '''
    Learning related constants
    '''
    MIN_EXPLORE_RATE = 0.001
    MIN_LEARNING_RATE = 0.2
    DECAY_FACTOR = np.prod(MAZE_SIZE, dtype=float) / 10.0

    '''
    Defining the simulation related constants
    '''
    NUM_EPISODES = 50000
    MAX_T = np.prod(MAZE_SIZE, dtype=int) * 100
    STREAK_TO_END = 100
    SOLVED_T = np.prod(MAZE_SIZE, dtype=int)
    DEBUG_MODE = 0
    RENDER_MAZE = True
    ENABLE_RECORDING = False

    '''
    Creating a Q-Table for each state-action pair
    '''
    q_table = np.zeros(NUM_BUCKETS + (NUM_ACTIONS,), dtype=float)

    '''
    Begin simulation
    '''
    recording_folder = "/tmp/maze_q_learning"
    """
    if ENABLE_RECORDING:
        env.monitor.start(recording_folder, force=True)
    """
    simulate()
    """
    if ENABLE_RECORDING:
        env.monitor.close()
    """

((13, 12), (9, 13))
test self.num_picks


SystemExit: 

d:\anaconda\envs\gym-final\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
